In [1]:
%reload_ext autoreload
%autoreload 2

## Dependencias

In [2]:
import pandas as pd 
import numpy as np 
import sys
sys.path.append("../")
import src.constants as c
from src.preprocessing import merge_similar_categories, CustomTransformer
from src.utils import save_to_pickle, read_pickle
from sklearn.preprocessing import StandardScaler

## Lectura de info.

In [3]:
X_train, y_train = read_pickle(c.train_data_v1)
X_test, y_test = read_pickle(c.test_data_v1)

In [4]:
numericas = read_pickle(c.numericas)
categoricas = read_pickle(c.categoricas)
drop = read_pickle(c.drop)

## Uniendo niveles

In [5]:
transformer = CustomTransformer(
    variables_to_eliminate=drop,
    discrete_variables=categoricas,
    continous_variables=numericas
)

In [6]:
transformer.fit(X_train, y_train)

CustomTransformer(continous_variables=['lat', 'sqft_living', 'sqft_living15',
                                       'sqft_above', 'sqft_lot', 'sqft_lot15',
                                       'sqft_basement', 'bathrooms',
                                       'bedrooms'],
                  discrete_variables=['zipcode', 'grade', 'floors', 'view',
                                      'waterfront'],
                  variables_to_eliminate=['condition', 'long', 'yr_built',
                                          'yr_renovated'])

In [7]:
X_train_transformed = transformer.transform(X_train, training=True)
y_train_masked = y_train[transformer.final_mask]

mask = (95000.000000 <= y_train_masked) & (y_train_masked <= 3400000.000000)

X_train_transformed = X_train_transformed[mask]
y_train_masked = y_train_masked[mask]

display(X_train_transformed.head())

percentage = len(X_train_transformed) / len(X_train) * 100
print(f"Percentage of kept rows: {percentage :.2f}%")

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,sqft_basement,lat,sqft_living15,sqft_lot15,zipcode_1,...,grade_5,grade_6,grade_7,floors_1,floors_2,floors_3,view_1,view_2,view_3,waterfront_1
6181,4.0,1.75,2090.0,7416.0,1050.0,1040.0,47.4107,1710.0,7527.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4688,3.0,2.50,1450.0,5175.0,1030.0,420.0,47.7082,1740.0,9250.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1703,3.0,1.75,3020.0,360241.0,3020.0,0.0,47.2662,1890.0,209959.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
21200,3.0,1.75,1370.0,1990.0,1370.0,0.0,47.6434,1730.0,1990.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
18874,3.0,1.75,1330.0,7216.0,1330.0,0.0,47.7199,1500.0,8000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Percentage of kept rows: 98.80%


In [8]:
transformer.dict_merged_levesl

{'zipcode': {(98002, 98032, 98168): 0,
  (98001,
   98003,
   98022,
   98023,
   98030,
   98031,
   98055,
   98148,
   98178,
   98188,
   98198): 1,
  (98042, 98092, 98106): 2,
  (98038, 98058, 98108, 98146): 3,
  (98133,): 4,
  (98118, 98126): 5,
  (98010,
   98011,
   98014,
   98019,
   98028,
   98045,
   98056,
   98070,
   98125,
   98155,
   98166): 6,
  (98034, 98059, 98065, 98136): 7,
  (98072, 98103, 98107, 98117): 8,
  (98007,
   98008,
   98024,
   98027,
   98029,
   98052,
   98077,
   98115,
   98116,
   98122,
   98144,
   98177): 9,
  (98053, 98074): 10,
  (98005, 98006, 98033, 98075, 98102, 98105, 98109, 98119, 98199): 11,
  (98112,): 12,
  (98004, 98040): 13,
  (98039,): 14},
 'grade': {(1, 3, 4, 5): 0,
  (6,): 1,
  (7,): 2,
  (8,): 3,
  (9,): 4,
  (10,): 5,
  (11,): 6,
  (12, 13): 7},
 'floors': {(1.0, 3.5): 0, (1.5, 3.0): 1, (2.0,): 2, (2.5,): 3},
 'view': {(0,): 0, (1, 2): 1, (3,): 2, (4,): 3},
 'waterfront': {(0,): 0, (1,): 1}}

In [9]:
for feature, table in transformer.dict_tables.items():
    print("="*120)
    print(f"Merged levels for {feature}")
    print("="*120)
    display(table)

Merged levels for zipcode


,mean_y,variance_y,n
"(98002, 98032, 98168)",2.419016e+05,2.184602e+09,544
"(98001, 98003, 98022, 98023, 98030, 98031, 98055, 98148, 98178, 98188, 98198)",2.964655e+05,2.176386e+09,2609
"(98042, 98092, 98106)",3.205848e+05,2.657636e+09,1115
"(98038, 98058, 98108, 98146)",3.581650e+05,8.967649e+09,1360
"(98133,)",3.862030e+05,1.003083e+10,446
"(98118, 98126)",4.153921e+05,1.003537e+10,773
"(98010, 98011, 98014, 98019, 98028, 98045, 98056, 98070, 98125, 98155, 98166)",4.469138e+05,6.658178e+09,2473
"(98034, 98059, 98065, 98136)",5.199062e+05,1.133238e+10,1424
"(98072, 98103, 98107, 98117)",5.760300e+05,1.158107e+10,1532
"(98007, 98008, 98024, 98027, 98029, 98052, 98077, 98115, 98116, 98122, 98144, 98177)",6.285599e+05,1.353774e+10,3430


Merged levels for grade


,mean_y,variance_y,n
"(1, 3, 4, 5)",2.449949e+05,1.266884e+10,244
"(6,)",3.020693e+05,1.450990e+10,1831
"(7,)",4.026546e+05,2.436286e+10,8108
"(8,)",5.408056e+05,4.688228e+10,5443
"(9,)",7.716995e+05,9.981974e+10,2353
"(10,)",1.075007e+06,2.340355e+11,1026
"(11,)",1.486759e+06,4.904660e+11,353
"(12, 13)",2.361449e+06,7.375416e+11,93


Merged levels for floors


,mean_y,variance_y,n
"(1.0, 3.5)",4.415483e+05,8.573495e+08,9585
"(1.5, 3.0)",5.631298e+05,2.743426e+10,2276
"(2.0,)",6.470905e+05,1.857011e+11,7443
"(2.5,)",1.048870e+06,7.345429e+11,147


Merged levels for view


,mean_y,variance_y,n
"(0,)",4.960342e+05,8.247778e+10,17539
"(1, 2)",7.976246e+05,2.051028e+11,1171
"(3,)",9.623470e+05,3.850071e+11,455
"(4,)",1.442562e+06,8.623071e+11,286


Merged levels for waterfront


,mean_y,variance_y,n
"(0,)",5.308489e+05,1.159312e+11,19306
"(1,)",1.626434e+06,1.298163e+12,145


## Estandarización

In [10]:
scaler = StandardScaler()

cols = transformer.continous_variables
X_train_transformed[cols] = (
    pd.DataFrame(
        scaler.fit_transform(X_train_transformed[cols]), 
        columns=cols,
        index=X_train_transformed.index
    )
)

## Guardando información

In [11]:
save_to_pickle([X_train_transformed, y_train_masked], c.train_data_v2)
save_to_pickle(transformer, c.transformer)
save_to_pickle(scaler, c.scaler)